In [0]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import numpy as np
import time, random
import matplotlib.pyplot as plt
from google.colab import files
import warnings

In [0]:
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)

In [0]:
#DATASET UPLOADS
upload = files.upload()

In [0]:
#NECESSARY TO ACCESS UPLOADED FILES, IDK WHY
for k,v in uploaded.keys():
  open(k,'wb').write(v)

In [0]:
training_images = np.load('emnist_digits_training_images.npy')
training_labels = np.load('emnist_digits_training_labels.npy')
test_images = np.load('emnist_digits_test_images.npy')
test_labels = np.load('emnist_digits_test_labels.npy')

In [0]:
def loss(logits,labels):
	labels = tf.reshape(tf.cast(labels,tf.int64),[-1])
	cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=logits,name='cross_entropy_per_example')
  cross_entropy_mean = tf.reduce_mean(cross_entropy,name='mean_ce')
  total_loss = tf.add(tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)),cross_entropy_mean,name='total_loss')
	
  return total_loss

def accuracy(logits,true_labels):
	pred_labels = tf.argmax(logits,1)
	true_labels = tf.cast(true_labels,tf.int64)
	#print pred_labels.get_shape().as_list(),true_labels
	correct_pred = tf.cast(tf.equal(pred_labels, true_labels), tf.float32)
	accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
	return accuracy

In [0]:
x = tf.placeholder(tf.float32,shape=[None,28,28],name='x')
y = tf.placeholder(tf.int64,shape=[None],name='y')
keep_prob = tf.placeholder(tf.float32,shape=(),name='keep_prob')
#lr = tf.placeholder(tf.float32,shape=())
mode = tf.placeholder(tf.bool,shape=(),name='mode')

aug_img = tf.placeholder(tf.float32,shape=[28,28],name='aug_img')

In [0]:
img_rotate = tf.contrib.image.rotate(aug_img,-0.5+np.random.random(),interpolation='BILINEAR')
img_affine = tf.contrib.image.transform(aug_img,[np.random.random() for i in xrange(8)])

In [0]:
#RESNET MODULE
def resmod(name,xinput,numfilts,s,projection):
  with tf.variable_scope(name):
    #1x1
    bn1 = tf.layers.batch_normalization(xinput,axis=3,training=mode,name=name+'bn1')
    act1 = tf.nn.relu(bn1,name=name+'relu1')
    conv1 = tf.layers.conv2d(act1,filters=numfilts,kernel_size=1,strides=1,padding='same',activation=None,kernel_initializer=tf.contrib.layeres.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005),dtype=tf.float32),name=name+'conv1_1x1')
    #3x3
    bn2 = tf.layers.batch_normalization(conv1,axis=3,training=mode,name=name+'bn2')
    act2 = tf.nn.relu(bn2,name=name+'relu2')
    conv2 = tf.layers.conv2d(act2,filters=numfilts,kernel_size=3,strides=s,padding='same',activation=None,kernel_initializer=tf.contrib.layeres.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005),dtype=tf.float32),name=name+'conv2_3x3')
    #1x1
    bn3 = tf.layers.batch_normalization(conv2,axis=3,training=mode,name=name+'bn3')
    act3 = tf.nn.relu(bn3,name=nae+'relu3')
    conv3 = tf.layers.conv2d(act3,filters=4*numfilts,kernel_size=1,strides=1,padding='same',activation=None,kernel_initializer=tf.contrib.layeres.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005),dtype=tf.float32),name=name+'conv3_1x1')
    
    #SHORTCUT
    if projection or s==2:
      shortcut = tf.layers.conv2d(xinput,filters=4*numfilts,kernel_size=1,strides=s,padding='same',activation=None,kernel_initializer=tf.contrib.layeres.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005),dtype=tf.float32),name=name+'shortcut')
    else:
      shortcut = xinput
      
    res_block_out = tf.add(conv3,shortcut,name=name+'output')
  
  return res_block_out

In [0]:
#RESNET V2
with tf.device('/gpu:0'):
  #28X28X3
  conv1 = tf.layers.conv2d(tf.reshape(x,shape=[-1,28,28,1]),filters=64,kernel_size=3,strides=1,padding='same',activation=tf.nn.relu,kernel_initializer=tf.contrib.layeres.xavier_initializer(uniform=False),kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005),dtype=tf.float32),name='conv1')
  pool1 = tf.layers.max_pooling2d(conv1,pool_size=3,strides=1,padding='same',name='pool1')
  #28x28x64
  res1_1 = resmod('res1_1',pool1,64,1,True)
  res1_2 = resmod('res1_2',res1_1,64,1,False)
  res1_3_= resmod('res1_3',res1_2,64,1,False)
  #28X28X256
  res2_1 = resmod('res2_1',res1_3,128,2,True)
  #14X14X512
  res2_2 = resmod('res2_2',res2_1,128,1,False)
  res2_3 = resmod('res2_3',res2_2,128,1,False)
  #res2_4 = resmod('res2_4',res2_3,128,1,False)
  #14X14X512
  res3_1 = resmod('res3_1',res2_3,256,2,True)
  #7X7X1024
  res3_2 = resmod('res3_2',res3_1,256,1,False)
  res3_3 = resmod('res3_3',res3_2,256,1,False)
  #res3_4 = resmod('res3_4',res3_3,256,1,False)
  #res3_5 = resmod('res3_5',res3_4,256,1,False)
  #res3_6 = resmod('res3_6',res3_5,256,1,False)
  #7X7X1024
  res4_1 = resmod('res4_1',res3_3,512,2,True)
  #3X3X2048
  res4_2 = resmod('res4_2',res4_1,512,1,False)
  res4_3 = resmod('res4_3',res4_2,512,1,False)
  #3X3X2048
  relu1 = t.nn.relu(res4_3,name='relu1')
  avg_pool = tf.layers.average_pooling2d(relu1,pool_size=3,strides=1,padding='valid',name='avg_pool')
  drouput1 = tf.layers.dropout(avg_pool,rate=keep_prob,training=mode)
  logits = tf.layers.dense(dropout1,units=10)
  
  cost = loss(logits,y)
  
  train_op = tf.train.AdamOptimizer(learning_rate=0.0001)
  
  update_ops = tf.get_collections(tf.GraphKeys.UPDATE_OPS)
  
  with tf.control_dependencies(update_ops):
    opt_op = train_op.minimize(cost)
  
  acc = accuracy(logits,y)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
epochs = 25
ne = 0
tracc=[]
valacc=[]
trerr=[]
valerr=[]
batch_size = 100
numiter = 2000
index=np.arange(200000)
index_t = np.arange(40000)

with tf.device('/gpu:0'):
  while(ne<epochs):
    print 'Epoch:: ',ne+1,'-->'
    stime = time.time()
    if ne != 0:
		  np.random.shuffle(index)
		  images = images[index]
		  labels = labels[index]
    for niter in xrange(numiter):
      #if (niter+1)%100==0:
        #print 'Iter::',niter+1,'---------'
      offset = niter*batch_size
      x_iter, y_iter = np.array(images[offset:offset+batch_size,:,:]), np.array(labels[offset:offset+batch_size])
      
      #Training Dict
      feed_trdict={x:x_iter,y:y_iter,keep_prob:0.8,mode:True}
		  
      #Train
      sess.run(opt_op,feed_dict=feed_trdict)
    
    
    #Calculate accuracy of Training set
    print 'Calculating Loss and Accuracy...'
    tr_loss = sess.run(cost,feed_dict=feed_trdict)
    tr_acc = sess.run(acc,feed_dict = feed_trdict)
    
    #Calculate accuracy of Validation set
    np.random.shuffle(index_t)
    val_loss = 0
    val_acc = 0
    for j in xrange(400):
      val_loss += sess.run(cost,feed_dict = {x:test_imgs[index_t[j*100:(j+1)*100]],y:test_labels[index_t[j*100:(j+1)*100]],keep_prob:1.0,mode:False})
      val_acc += sess.run(acc,feed_dict = {x:test_imgs[index_t[j*100:(j+1)*100]],y:test_labels[index_t[j*100:(j+1)*100]],keep_prob:1.0,mode:False})
    val_loss/=400
    val_acc/=400
    print 'Appending values'
    valacc.append(val_acc)
    tracc.append(tr_acc)
    trerr.append(tr_loss)
    valerr.append(val_loss)
    print 'Epoch..',ne+1,'...'
    print 'Training cost::',tr_loss,' Validation Cost:: ',val_loss
    print 'Training accuracy::',tr_acc*100,'%',' Validation accuracy::',val_acc*100,'%'
    print 'Time reqd.::',(time.time()-stime)/60,'mins...'
    ne = ne + 1

In [0]:
plt.plot(tracc)
plt.plot(valacc)
plt.show()

In [0]:
plt.plot(trerr)
plt.plot(valerr)
plt.show()

In [0]:
#INFERENCE
test_acc=[]
tindex = np.arange(40000)

for nepoch in xrange(100):
  stime = time.time()
	if nepoch !=0:
		np.random.shuffle(tindex)
		test_x = test_x[index]
		test_y = test_y[index]
	test_loss = sess.run(cost,feed_dict = {feed_images:test_x,feed_labels:test_y,keep_prob:1.0,mode=False})
	test_acc = sess.run(acc,feed_dict = {feed_images:test_x,feed_labels:test_y,keep_prob:1.0,mode=False})
	print 'Epoch..',ne+1,'...'
	print 'Time reqd.::',(time.time()-stime)/60,'mins...'
  test_acc.append(test_acc)

In [0]:
plt.plot(test_acc)

In [0]:
#SAVE MODEL
graph = tf.get_default_graph()

with graph.as_default():
  input_dict = {"x":x,"y":y,"keep_prob":keep_prob,"aug_img":aug_img,"mode":mode}
  output_dict = {"logits":logits,"aux_logits":aux_logits}
  tf.saved_model.simple_save(sess,'/mnist_saved_model/',input_dict,output_dict)

In [0]:
sess.close()